In [ ]:
pip install --upgrade tensorflow==2.15 transformers rouge_score datasets accelerate>=0.20.1 evaluate


In [ ]:
pip install --upgrade tensorflow-probability


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 19.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-probability
    Found existing installation: tensorflow-probability 0.23.0
    Uninstalling tensorflow-probability-0.23.0:
      Successfully uninstalled tensorflow-probability-0.23.0


LOAD DATASET

In [ ]:
!wget https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Dataset.json

--2024-06-04 17:53:50--  https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34667 (34K) [text/plain]
Saving to: ‘Dataset.json’

Dataset.json        100%[===================>]  33.85K  --.-KB/s    in 0.007s  

2024-06-04 17:53:50 (4.98 MB/s) - ‘Dataset.json’ saved [34667/34667]



In [ ]:
!wget https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Train_Dataset.json

--2024-06-04 17:53:52--  https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Train_Dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 137567 (134K) [text/plain]
Saving to: ‘Train_Dataset.json’

Train_Dataset.json  100%[===================>] 134.34K  --.-KB/s    in 0.02s   

2024-06-04 17:53:52 (6.20 MB/s) - ‘Train_Dataset.json’ saved [137567/137567]



In [ ]:
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import (AutoTokenizer, DataCollatorForSeq2Seq,
                          AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer)

In [ ]:
my_dataset_test = load_dataset("json", data_files="Dataset.json", field="documents")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
my_dataset_train = load_dataset("json", data_files="Train_Dataset.json", field="documents")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
my_dataset_train

DatasetDict({
    train: Dataset({
        features: ['TEXT', 'SUMMARIZATION'],
        num_rows: 95
    })
})

PREPROCESS

In [ ]:
checkpoint = "facebook/bart-large-cnn"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["TEXT"]]
    #model_inputs = tokenizer(inputs, padding="max_length", truncation=True, return_tensors="pt")
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    #labels = tokenizer(examples["SUMMARIZATION"], padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(text_target=examples["SUMMARIZATION"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
my_dataset_test = my_dataset_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

In [ ]:
my_dataset_test

DatasetDict({
    train: Dataset({
        features: ['TEXT', 'SUMMARIZATION', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 25
    })
})

In [ ]:
my_dataset_train = my_dataset_train.map(preprocess_function, batched=True)

Map:   0%|          | 0/95 [00:00<?, ? examples/s]

CREATING BATCHING

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

TRAIN

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_controls",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=3,
    predict_with_generate= False,
    fp16=False,
    push_to_hub=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=my_dataset_train["train"],
    eval_dataset=my_dataset_test["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.153084
2,No log,2.067945
3,No log,2.436864


TrainOutput(global_step=285, training_loss=0.8435953776041667, metrics={'train_runtime': 43.0413, 'train_samples_per_second': 6.622, 'train_steps_per_second': 6.622, 'total_flos': 140590911062016.0, 'train_loss': 0.8435953776041667, 'epoch': 3.0})

TrainOutput(global_step=285, training_loss=0.8559526945415296, metrics={'train_runtime': 37.8825, 'train_samples_per_second': 7.523, 'train_steps_per_second': 7.523, 'total_flos': 140590911062016.0, 'train_loss': 0.8559526945415296, 'epoch': 3.0})

SAVE MODEL

In [ ]:
model.save_pretrained("my_controls")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [ ]:
tokenizer.save_pretrained("my_controls")

('my_controls/tokenizer_config.json',
 'my_controls/special_tokens_map.json',
 'my_controls/vocab.json',
 'my_controls/merges.txt',
 'my_controls/added_tokens.json',
 'my_controls/tokenizer.json')

INFERENCE

1 WAY

In [ ]:
text = "summarize: A Standard Change is created from preapproved Standard change templates that have satisfied specific criteria (see Appendix A) and been added to the Standard Change Template Catalog in the Enterprise Ticketing System: the change is repeatable, frequently implemented, is considered low risk and low impact according to the SIA, and has a proven history of success (completed the Normal change lifecycle process at least 3-5 times with no issues). Standard changes that are approved by Division Directors and CAB Co-Chairs are added to the catalog and considered pre-authorized, following a shorter ChM lifecycle outside of the CAB approval process (subject to dual authorization). CRs can request new Standard change templates or use an existing template from the catalog to create a new Standard change request.A Normal Change is one that meets the defined lead time for testing and validation and is assigned a SIA level of no, low, medium, or high. A Normal change is an RFC that is not a Standard, Expedited or Emergency change, and is subject to the full ChM review process, including review and authorization by the CAB.An Expedited Change does not meet the lead time requirement for a Normal change, but is not an Emergency Change. Service is at risk, although service might not be down, and the RFC needs to be authorized because of a client request that has been validated by SME/ technical expert or a Director, who has determined that that the change needs to go in without waiting for the recommended lead-time. The same ‘Normal’ Change request information is provided in the Enterprise Ticketing System to implement the change, including the reason for expediting the RFC (SIA, back-out plans, scheduled time and downtime required), but lead times are much shorter. Authorization by a CAB Co-Chair is required and Expedited Changes are subject to retroactive review by CAB. Emergency Change is one that must be implemented as soon as possible to correct, or prevent, a high priority incident, or that must be introduced as soon as possible due to likely negative service impacts or situations where the impact to a service is imminent if action is not taken. These changes do not follow the complete lifecycle of a Normal change due to the speed with which they must be implemented and authorized. All emergency changes are authorized by E-CAB and documented and entered into the Enterprise Ticketing System prior to implementation, or as soon as possible after the change has been implemented depending on the nature of the emergency.Emergency changes are subject to a Post Implementation Review (PIR) process by CAB."


In [ ]:
text_prompt ="""A Standard Change is created from preapproved Standard change templates that have satisfied specific criteria (see Appendix A) and been added to the Standard Change Template Catalog in the Enterprise Ticketing System: the change is repeatable, frequently implemented, is considered low risk and low impact according to the SIA, and has a proven history of success (completed the Normal change lifecycle process at least 3-5 times with no issues). Standard changes that are approved by Division Directors and CAB Co-Chairs are added to the catalog and considered pre-authorized, following a shorter ChM lifecycle outside of the CAB approval process (subject to dual authorization). CRs can request new Standard change templates or use an existing template from the catalog to create a new Standard change request.A Normal Change is one that meets the defined lead time for testing and validation and is assigned a SIA level of no, low, medium, or high. A Normal change is an RFC that is not a Standard, Expedited or Emergency change, and is subject to the full ChM review process, including review and authorization by the CAB.An Expedited Change does not meet the lead time requirement for a Normal change, but is not an Emergency Change. Service is at risk, although service might not be down, and the RFC needs to be authorized because of a client request that has been validated by SME/ technical expert or a Director, who has determined that that the change needs to go in without waiting for the recommended lead-time. The same ‘Normal’ Change request information is provided in the Enterprise Ticketing System to implement the change, including the reason for expediting the RFC (SIA, back-out plans, scheduled time and downtime required), but lead times are much shorter. Authorization by a CAB Co-Chair is required and Expedited Changes are subject to retroactive review by CAB. Emergency Change is one that must be implemented as soon as possible to correct, or prevent, a high priority incident, or that must be introduced as soon as possible due to likely negative service impacts or situations where the impact to a service is imminent if action is not taken. These changes do not follow the complete lifecycle of a Normal change due to the speed with which they must be implemented and authorized. All emergency changes are authorized by E-CAB and documented and entered into the Enterprise Ticketing System prior to implementation, or as soon as possible after the change has been implemented depending on the nature of the emergency.Emergency changes are subject to a Post Implementation Review (PIR) process by CAB.
Review by this
Summary:
"""

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="my_controls")
summarizer(text)

[{'summary_text': 'A Normal Change is one that meets the defined lead time for testing and validation and is assigned a SIA level of no, low, medium, or high. An Expedited Change does not meet the lead time requirement for a Normal change, but is not an Emergency Change. Emergency changes are subject to a Post Implementation Review (PIR) process by CAB.'}]

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="my_controls")
summarizer(text_prompt)

[{'summary_text': 'A Normal Change is one that meets the defined lead time for testing and validation and is assigned a SIA level of no, low, medium, or high. A Normal change is an RFC that is not a Standard, Expedited or Emergency change, and is subject to the full ChM review process, including review and authorization by the CAB.'}]

2 WAY


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my_controls")
inputs = tokenizer(text, return_tensors="pt").input_ids

from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("my_controls")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)

tokenizer.decode(outputs[0], skip_special_tokens=True)

'A Normal Change is one that meets the defined lead time for testing and validation and is assigned a SIA level of no, low, medium, or high. An Expedited Change does not meet the lead time requirement for a Normal change, but is not an Emergency Change. All emergency changes are authorized by E-CAB and documented and entered into the Enterprise Ticketing System prior to implementation.'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my_controls")
inputs = tokenizer(text_prompt, return_tensors="pt").input_ids

from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("my_controls")
outputs = model.generate(inputs, max_new_tokens=100, do_sample=False)

tokenizer.decode(outputs[0], skip_special_tokens=True)

'A Normal Change is one that meets the defined lead time for testing and validation and is assigned a SIA level of no, low, medium, or high. A Normal change is an RFC that is not a Standard, Expedited or Emergency change, and is subject to the full ChM review process, including review and authorization by the CAB.'

ΔΗΜΙΟΥΡΓΙΑ ΛΙΣΤΑΣ ΜΕ ΑΠΟΤΕΛΕΣΜΑΤΑ ΤΟΥ ΜΟΝΤΕΛΟΥ MY_CONTROLS ΑΠΟ ΠΡΟΣΠΕΛΑΣΗ ΤΟΥ DATASET TEST

In [ ]:
i=0
results_dict = {}
!wget https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Dataset.json
my_dataset_test = load_dataset("json", data_files="Dataset.json", field="documents")


summarizer = pipeline("summarization", model="my_controls")

for i in range(len(my_dataset_test["train"])):

  data_item = my_dataset_test["train"][i]
  text = data_item.get("TEXT")
  text_with_prompt = "Review by this" + text

  answer = summarizer(text_with_prompt)


  results_dict[i] = answer

print(results_dict)

--2024-06-04 19:14:10--  https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34667 (34K) [text/plain]
Saving to: ‘Dataset.json.13’

Dataset.json.13     100%[===================>]  33.85K  --.-KB/s    in 0.008s  

2024-06-04 19:14:10 (4.28 MB/s) - ‘Dataset.json.13’ saved [34667/34667]



Your max_length is set to 142, but your input_length is only 105. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=52)
Your max_length is set to 142, but your input_length is only 100. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 142, but your input_length is only 109. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=54)
Your max_length is set to 142, but your input_length is only 109. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=54)


{0: [{'summary_text': 'Data is to be given appropriate levels of access control and security. This means that it should be safeguarded by means of lockable cabinets and password and/or encryption protection. Handling and exchange of information must in addition comply with the Access to Health Records Act 1990. Only those with a professional or contractual duty of confidentiality are permitted access.'}], 1: [{'summary_text': 'Data is to be given appropriate levels of access control and security. This means that it should be safeguarded by means of lockable cabinets and password and/or encryption protection. Handling and exchange of information must in addition comply with the Access to Health Records Act 1990. Only those with a professional or contractual duty of confidentiality are permitted access.'}], 2: [{'summary_text': 'All members of the EUI community and visitors to the campus have a collective responsibility for adhering to the security procedures and measures in place. Heads

ΔΗΜΙΟΥΡΓΙΑ ΛΙΣΤΑΣ ΜΕ ΑΠΟΤΕΛΕΣΜΑΤΑ ROUGE

In [ ]:
pip install rouge-score
!wget https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Dataset.json
my_dataset = load_dataset("json", data_files="Dataset.json", field="documents")
my_dataset_test = my_dataset['train']
len(my_dataset_test)

from rouge_score import rouge_scorer
i=0

rouge_scores = []

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

for i in range(len(results_dict)):
    hypothesis = my_dataset_test['SUMMARIZATION'][i]

    reference = results_dict[i][0]['summary_text']
    if isinstance(hypothesis, str):
     hypothesis = hypothesis.lower()

    if isinstance(reference, str):
     reference = reference.lower()

    score = scorer.score(reference, hypothesis)

    rouge_scores.append(score)

for idx, score in enumerate(rouge_scores):
    print(f"Αποτέλεσμα για το ζεύγος {idx+1}: {score}")



Αποτέλεσμα για το ζεύγος 1: {'rouge1': Score(precision=0.3125, recall=0.08333333333333333, fmeasure=0.13157894736842105), 'rouge2': Score(precision=0.06666666666666667, recall=0.01694915254237288, fmeasure=0.02702702702702703), 'rougeL': Score(precision=0.25, recall=0.06666666666666667, fmeasure=0.10526315789473685)}
Αποτέλεσμα για το ζεύγος 2: {'rouge1': Score(precision=0.375, recall=0.1, fmeasure=0.15789473684210528), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.1875, recall=0.05, fmeasure=0.07894736842105264)}
Αποτέλεσμα για το ζεύγος 3: {'rouge1': Score(precision=0.48, recall=0.25, fmeasure=0.3287671232876712), 'rouge2': Score(precision=0.041666666666666664, recall=0.02127659574468085, fmeasure=0.028169014084507043), 'rougeL': Score(precision=0.24, recall=0.125, fmeasure=0.1643835616438356)}
Αποτέλεσμα για το ζεύγος 4: {'rouge1': Score(precision=0.3870967741935484, recall=0.25, fmeasure=0.3037974683544304), 'rouge2': Score(precision=0.033333

In [ ]:
print(rouge_scores[1])

{'rouge1': Score(precision=0.375, recall=0.1, fmeasure=0.15789473684210528), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.1875, recall=0.05, fmeasure=0.07894736842105264)}


In [ ]:
len(rouge_scores)

25

ΜΕΣΟΣ ΟΡΟΣ ΑΠΟ ΤΗΝ ΛΙΣΤΑ ROUGE_SCORES

In [ ]:
import numpy as np

all_rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

print(all_rouge_scores)
print(len(all_rouge_scores))

for score in rouge_scores:
    all_rouge_scores['rouge1'].append(score['rouge1'])
    all_rouge_scores['rouge2'].append(score['rouge2'])
    all_rouge_scores['rougeL'].append(score['rougeL'])

print(all_rouge_scores)
print(len(all_rouge_scores))

# Υπολογισμός Μ.Ο για το ROUGE-1
rouge1_array = np.array([[metric.precision, metric.recall, metric.fmeasure] for metric in all_rouge_scores['rouge1']])
rouge1_means = np.mean(rouge1_array, axis=0)
print("Μέσος όρος ROUGE1: Precision = {:.4f}, Recall = {:.4f}, F-measure = {:.4f}".format(rouge1_means[0], rouge1_means[1], rouge1_means[2]))

# Υπολογισμός Μ.Ο για το ROUGE-2
rouge2_array = np.array([[metric.precision, metric.recall, metric.fmeasure] for metric in all_rouge_scores['rouge2']])
rouge2_means = np.mean(rouge2_array, axis=0)
print("Μέσος όρος ROUGE2: Precision = {:.4f}, Recall = {:.4f}, F-measure = {:.4f}".format(rouge2_means[0], rouge2_means[1], rouge2_means[2]))

# Υπολογισμός Μ.Ο για το ROUGE-L
rougeL_array = np.array([[metric.precision, metric.recall, metric.fmeasure] for metric in all_rouge_scores['rougeL']])
rougeL_means = np.mean(rougeL_array, axis=0)
print("Μέσος όρος ROUGE-L: Precision = {:.4f}, Recall = {:.4f}, F-measure = {:.4f}".format(rougeL_means[0], rougeL_means[1], rougeL_means[2]))


{'rouge1': [], 'rouge2': [], 'rougeL': []}
3
{'rouge1': [Score(precision=0.3125, recall=0.08333333333333333, fmeasure=0.13157894736842105), Score(precision=0.375, recall=0.1, fmeasure=0.15789473684210528), Score(precision=0.48, recall=0.25, fmeasure=0.3287671232876712), Score(precision=0.3870967741935484, recall=0.25, fmeasure=0.3037974683544304), Score(precision=0.30434782608695654, recall=0.125, fmeasure=0.17721518987341772), Score(precision=0.4166666666666667, recall=0.17857142857142858, fmeasure=0.25), Score(precision=0.18181818181818182, recall=0.03389830508474576, fmeasure=0.05714285714285715), Score(precision=0.3103448275862069, recall=0.15254237288135594, fmeasure=0.20454545454545459), Score(precision=0.2857142857142857, recall=0.13559322033898305, fmeasure=0.1839080459770115), Score(precision=0.3076923076923077, recall=0.07142857142857142, fmeasure=0.11594202898550726), Score(precision=0.37037037037037035, recall=0.17857142857142858, fmeasure=0.24096385542168672), Score(precis

In [ ]:
!wget https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Dataset.json
my_dataset_test = load_dataset("json", data_files="Dataset.json", field="documents")
column1 = results_dict
column2 = my_dataset_test['train']['SUMMARIZATION']

import pandas as pd
from tabulate import tabulate
import textwrap

column1 = my_dataset_test['train']['TEXT']
column2 = list(results_dict.values())
column3 = my_dataset_test['train']['SUMMARIZATION']

if len(column1) != len(column2):
    raise ValueError("Οι στήλες δεν έχουν το ίδιο μήκος!")

max_text_width = 80
column1_wrapped = [textwrap.fill(text, width=max_text_width) for text in column1]
column2_wrapped = [textwrap.fill(str(text), width=max_text_width) for text in column2]
column3_wrapped = [textwrap.fill(text, width=max_text_width) for text in column3]

result_df = pd.DataFrame({
    'Column1': column1_wrapped,
    'Column2': column2_wrapped,
    'Column3': column3_wrapped
})
print(tabulate(result_df, headers='keys', tablefmt='grid', showindex=False))

--2024-06-04 19:13:42--  https://raw.githubusercontent.com/MarCaramani/MK_Thesis/master/THESIS/Dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34667 (34K) [text/plain]
Saving to: ‘Dataset.json.11’

Dataset.json.11     100%[===================>]  33.85K  --.-KB/s    in 0.007s  

2024-06-04 19:13:42 (4.69 MB/s) - ‘Dataset.json.11’ saved [34667/34667]

+----------------------------------------------------------------------------------+----------------------------------------------------------------------------------+----------------------------------------------------------------------------------+
| Column1                                                                          | Column2                                                            